# Automated download of data from NASA APPEEARS for USGS site/basin of interest and reformatting for use within ILAMB-Watershed

In [187]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json
import pandas as pd
import geopandas as gpd

In [190]:
pd.to_datetime("today").strftime("%Y%m%d")
proj

'geographic'

In [184]:
AOIFILE='/home/jbk/projects/climate/interface/watershed_intercomparison/ARW_for_nate/shape_files/ARW_watershed.geojson'

In [189]:
case_name=AOIFILE.split('/')[-1].split('.')[0]+"_"+pd.to_datetime("today").strftime("%Y%m%d")
case_name

'ARW_watershed_20230415'

# NASA APPEEARS Products list
https://appeears.earthdatacloud.nasa.gov/products

# NASA APPEEARS REST API tutorial
https://lpdaac.usgs.gov/resources/e-learning/getting-started-with-the-a%CF%81%CF%81eears-api-submitting-and-downloading-an-area-request/

In [97]:
# Set input directory, change working directory
inDir = os.getcwd()                                    # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                        # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

In [99]:
# Login with NASA Earthdata Account
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username:  ········
Enter NASA Earthdata Login Password:  ········


In [100]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response  

{'token_type': 'Bearer',
 'token': 'Rbf2fYON10KJz_AZyj5z7sYnGUuNKJjH2CWt6rLVN7gfUZ_CgEgCHNR0ZbhaqKIr9tUNYKG_DuI3y8T9VT9dBg',
 'expiration': '2023-04-17T17:34:09Z'}

In [101]:
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AρρEEARS currently supports {} products.'.format(len(product_response)))  # Print no. products available in AppEEARS

AρρEEARS currently supports 168 products.


In [102]:
all_products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version

In [103]:

products = ['MYD16A2.061', 'MOD16A2.061', 'MCD15A2H.061', 'MYD17A2.061']                                         # Print information for MYD16A2.061 ET Product

In [160]:
# Create list of products of interest
# MYD16A2.061  ET -- MODIS Aqua
# MOD16A2.061  ET -- MODIS Terra
# MCD15A2H.061 LAI -- combined MODIS
# MYD15A2H.061 LAI -- MODIS Aqua
# MOD15A2H.061 LAI -- MODIS Terra
# MYD17A2.061  GPP -- MODIS Aqua
# MOD17A2.061  GPP -- MODIS Terra

prod_list = ['MYD16A2.061', 'MOD16A2.061', 'MCD15A2H.061', 'MYD15A2H.061', 'MOD15A2H.061', 'MYD17A2H.061', 'MOD17A2H.061']
# Get the layers available from MYD16A2.061
print(r.get('{}product/{}'.format(api, prod_list[0])).json().keys())
# Get the layers available from MOD16A2.061
print(r.get('{}product/{}'.format(api, prod_list[1])).json().keys())

# Get the layers available from MCD15A2H.61
print(r.get('{}product/{}'.format(api, prod_list[2])).json().keys())
# Get the layers available from MYD15A2H.61
print(r.get('{}product/{}'.format(api, prod_list[3])).json().keys())
# Get the layers available from MOD15A2H.61
print(r.get('{}product/{}'.format(api, prod_list[4])).json().keys())

# Get the layers available from MYD17A2.061
print(r.get('{}product/{}'.format(api, prod_list[5])).json().keys())
# Get the layers available from MOD17A2.061
print(r.get('{}product/{}'.format(api, prod_list[6])).json().keys())



dict_keys(['ET_500m', 'ET_QC_500m', 'LE_500m', 'PET_500m', 'PLE_500m'])
dict_keys(['ET_500m', 'ET_QC_500m', 'LE_500m', 'PET_500m', 'PLE_500m'])
dict_keys(['FparExtra_QC', 'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m'])
dict_keys(['FparExtra_QC', 'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m'])
dict_keys(['FparExtra_QC', 'FparLai_QC', 'FparStdDev_500m', 'Fpar_500m', 'LaiStdDev_500m', 'Lai_500m'])
dict_keys(['Gpp_500m', 'PsnNet_500m', 'Psn_QC_500m'])
dict_keys(['Gpp_500m', 'PsnNet_500m', 'Psn_QC_500m'])


In [170]:

# Get list of layers for this product
# Add ET vars to the list of layers we want to request
# ET - MODIS Aqua
layers = [(prod_list[0],'ET_500m')]  # Create tupled list linking desired product with desired layers
# ET - MODIS Terra
layers.append((prod_list[1],'ET_500m'))

## Add LAI vars to the list of layers we want to request
# LAI - MODIS Combined
#layers.append((prod_list[2],'Lai_500m'))
# LAI - MODIS Aqua
#layers.append((prod_list[3],'Lai_500m'))
# LAI - MODIS Terra
#layers.append((prod_list[4],'Lai_500m'))

## Add GPP vars to the list of layers we want to request
# GPP - MODIS Aqua
#layers.append((prod_list[5],'Gpp_500m'))
# GPP - MODIS Terra
#layers.append((prod_list[6],'Gpp_500m'))

# Create a list of dictionaries 
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'ET_500m', 'product': 'MYD16A2.061'},
 {'layer': 'ET_500m', 'product': 'MOD16A2.061'}]

In [171]:
# Submit an Area Request
# The Submit task API call provides a way to submit a new request to be processed. It can accept data via JSON, query string, or a combination of both. In the example below, compile a json and submit a request. Tasks in AρρEEARS correspond to each request associated with your user account. Therefore, each of the calls to this service requires an authentication token (see Section 1c.), which is stored in a header below.
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

In [186]:
# Read a GeoJSON for the AOI
aoi = gpd.read_file(AOIFILE) # Read in GeoJSON or Shapefile as dataframe using geopandas
aoi = aoi.drop(columns=['loaddate']).explode()
aoi = json.loads(aoi.to_json())
#print(aoi.head())        # Print first few lines of dataframe


/tmp/ipykernel_569688/3215307557.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  aoi = aoi.drop(columns=['loaddate']).explode()


In [173]:
# Projections supported by APPEEARS
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
#projections 
# Create a dictionary of projections with projection Name as the keys.
projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north']

In [174]:
# Compile a JSON
task_name = 'modis_basin_07340300' # User-defined name of the task: 'NPS Vegetation Area' used in example
task_type = ['point', 'area']        # Type of task, area or point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['netcdf4']  # Set output file format type
startDate = '01-01-2000'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '12-31-2022'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

# create JSON
task = {
    'task_type': task_type[1],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate,
             'recurring': False,
             'yearRange': [1950, 2050],
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': aoi,
    }
}

In [175]:
# Submit a Task Request
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response  

{'task_id': '37da1fbe-03fb-4353-b442-8f3e743e355f', 'status': 'pending'}

In [176]:
# Retrieve Task Status
params = {'limit': 2, 'pretty': True} # Limit API response to 2 most recent entries, return as pretty json
tasks_response = r.get('{}task'.format(api), params=params, headers=head).json() # Query task service, setting params and header 
tasks_response     



[{'tier': 2,
  'error': None,
  'params': {'dates': [{'endDate': '12-31-2022',
     'recurring': False,
     'startDate': '01-01-2000',
     'yearRange': [1950, 2050]}],
   'layers': [{'layer': 'ET_500m', 'product': 'MYD16A2.061'},
    {'layer': 'ET_500m', 'product': 'MOD16A2.061'}],
   'output': {'format': {'type': 'netcdf4'}, 'projection': 'geographic'}},
  'status': 'done',
  'crashed': False,
  'created': '2023-04-15T22:43:37.814249',
  'task_id': '37da1fbe-03fb-4353-b442-8f3e743e355f',
  'updated': '2023-04-15T22:47:27.907066',
  'user_id': 'jitendra.ornl@gmail.com',
  'attempts': 1,
  'estimate': {'request_size': 187145844.33801442, 'request_memory': 1},
  'retry_at': None,
  'completed': '2023-04-15T22:47:27.891196',
  'has_swath': False,
  'task_name': 'modis_basin_07340300',
  'task_type': 'area',
  'api_version': 'v1',
  'svc_version': '3.27',
  'web_version': None,
  'has_nsidc_daac': False,
  'expires_on': '2023-05-15T22:47:27.907066'},
 {'tier': 2,
  'error': None,
  'para

In [177]:
# take the task id returned from the task_response that was generated when submitting your request, and use the AρρEEARS API status service to check the status of your request.
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
status_response          

{'tier': 2,
 'error': None,
 'params': {'geo': {'type': 'FeatureCollection',
   'features': [{'id': '(0, 0)',
     'type': 'Feature',
     'geometry': {'type': 'Polygon',
      'coordinates': [[[-121.35821784706809, 47.14009586016249],
        [-121.35773137727716, 47.1400293601626],
        [-121.35717221477802, 47.140057053912585],
        [-121.35663232519556, 47.1400255007876],
        [-121.35630024290441, 47.1400445445376],
        [-121.35575595019691, 47.140155777870746],
        [-121.35524396478104, 47.14035080599547],
        [-121.35485515644831, 47.14051188828688],
        [-121.35448598769887, 47.140601821620066],
        [-121.35403218978291, 47.14060720287006],
        [-121.35368486999181, 47.14055459245344],
        [-121.35330597728404, 47.14039438516204],
        [-121.35281089082645, 47.14004202787095],
        [-121.35233141374385, 47.13974941745471],
        [-121.35178439082807, 47.139384405996964],
        [-121.35146518874524, 47.1389869549559],
        [-121.

In [179]:
# call the task service for your request every 20 seconds to check the status of your request.
# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

done


In [180]:
# Download a Request
destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

# Explore Files in Request Output 
# The bundle API provides information about completed tasks. For any completed task, a bundle can be queried to return the files contained as a part of the task request. Below, call the bundle API and return all of the output files.
bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
bundle

# Download Files in a Request (Automation)
# use the contents of the bundle to select the file name and id and store as a dictionary.
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files 

# Use the files dictionary and a for loop to automate downloading all of the output files into the output directory.
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
    if files[f].endswith('.tif'):
        filename = files[f].split('/')[1]
    else:
        filename = files[f] 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))


Downloaded files can be found at: /home/jbk/projects/climate/interface/src/hydrology_codes/modis_basin_07340300


In [73]:
bundle

{'files': [{'sha256': '2b1048abce3a6eed0ba4b8f63a1d8bd38d43e4e03598d921ab10e7fbae394a83',
   'file_id': 'd2223a2c-d057-4328-ac4d-0117203c53b0',
   'file_name': 'modis-basin-07340300-request.json',
   'file_size': 6774,
   'file_type': 'json',
   's3_url': 's3://appeears-output/56d7d841-a043-4ce9-9263-f691e7c61d7f/modis-basin-07340300-request.json'},
  {'sha256': 'ac81e9135a2465e532192121ce9722a3b29cffe7955ff5faf11b7a2585b72290',
   'file_id': '4ebe17b6-9c4d-449e-8323-016d90fa05f7',
   'file_name': 'modis-basin-07340300-MCD15A2H-061-metadata.xml',
   'file_size': 21090,
   'file_type': 'xml',
   's3_url': 's3://appeears-output/56d7d841-a043-4ce9-9263-f691e7c61d7f/modis-basin-07340300-MCD15A2H-061-metadata.xml'},
  {'sha256': 'ebe0dad86f6aaedd2ec48bbc3f85306eac83224959bbbbc6fae47b5c5754de49',
   'file_id': 'daf2774e-58ba-4a99-825f-9dc63bd511f6',
   'file_name': 'modis-basin-07340300-MYD16A2-061-metadata.xml',
   'file_size': 21022,
   'file_type': 'xml',
   's3_url': 's3://appeears-outpu

In [76]:
aoi_1 = gpd.read_file(AOIFILE) # Read in GeoJSON or Shapefile as dataframe using geopandas


In [77]:
aoi_1.area

/tmp/ipykernel_569688/2240628493.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_1.area


0    0.024761
dtype: float64

In [94]:
aoi_1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [92]:
aoi_1.to_crs(3395).area/10e6

0    37.028946
dtype: float64

In [122]:
task

{'task_type': 'area',
 'task_name': 'modis_basin_07340300',
 'params': {'dates': [{'startDate': '01-01-2018',
    'endDate': '12-31-2018',
    'recurring': 'false'}],
  'layers': [{'layer': 'ET_500m', 'product': 'MYD16A2.061'},
   {'layer': 'PET_500m', 'product': 'MYD16A2.061'},
   {'layer': 'LAI_500m', 'product': 'MCD15A2H.061'},
   {'layer': 'GPP_500m', 'product': 'MYD17A2H.061'}],
  'output': {'format': {'type': 'netcdf4'}, 'projection': 'geographic'},
  'geo': {'type': 'FeatureCollection',
   'features': [{'id': '0',
     'type': 'Feature',
     'properties': {'objectid': 12817.0,
      'tnmid': '{81026153-FC9A-4685-BE63-BAAF48D328B9}',
      'metasource': None,
      'sourcedata': None,
      'sourceorig': None,
      'sourcefeat': None,
      'referenceg': None,
      'areaacres': 219886.64,
      'areasqkm': 889.85,
      'states': 'WA',
      'huc10': '1703000201',
      'name': 'Little Naches River',
      'hutype': 'S',
      'humod': None,
      'globalid': '{1B73A39F-E29C-1